In [1]:
class backtest():
    def __init__(self,data_out,price):
        self.data_out=data_out.dropna().iloc[::-1]#late to the last
        self.price=price.head(len(self.data_out)).iloc[::-1]
    def backtest(self,trade_size=100,ifshort=True):
        prev_x_qty = 0 #amount
        position = 0 #-1,0,1
        trade_size = trade_size
        signal = np.sign(self.data_out.Signal)
        self.data_out['buyOrsell']=signal
        qty_x = pd.Series(np.zeros(len(signal)),index=self.data_out.index) #money spend
        
        if ifshort:
            for i in range(len(signal)) :
              if signal[i] == 1 and position == 0 :
                # buy the spread initially
                prev_x_qty = trade_size
                qty_x[i] = prev_x_qty
                position = 1


              if signal[i] == -1 and position == 0 :
                # sell the spread initially
                prev_x_qty =  trade_size
                qty_x[i] = -prev_x_qty
                position = -1



              if signal[i] == 1 and position == -1 :
                # we were short the spread and need to buy
                qty_x[i] = trade_size + prev_x_qty#新头寸+平旧头寸
                prev_x_qty = trade_size
                position = 1


              if signal[i] == -1 and position == 1 :
                # we are long the spread and need to sell
                qty_x[i] = -(trade_size + prev_x_qty)
                prev_x_qty = trade_size
                position = -1
                
        else:
            for i in range(len(signal)) :
              if signal[i] == 1 and position == 0 :
                # buy the spread initially
                qty_x[i] = trade_size
                position = 1


              if signal[i] == -1 and position == 0 :
                # sell the spread initially
                qty_x[i] = 0
                position = 0

              if signal[i] == -1 and position == 1 :
                # we are long the spread and need to sell
                qty_x[i] = - trade_size 
                position = 0
            
        qty_x[len(qty_x)-1] = -sum(qty_x)#zero out
        self.data_out["qty_x"] = qty_x#对x交易行为序列
        
        cash_buy=[(np.sign(q)==1)*q*self.price[i] for i,q in enumerate(self.data_out.qty_x) ]
        cash_sell=[-((np.sign(q)==-1)*q*self.price[i]) for i,q in enumerate(self.data_out.qty_x) ]
        pos = np.cumsum(self.data_out.qty_x)
        cumulative_buy = np.cumsum(cash_buy)#现金支出
        cumulative_sell = np.cumsum(cash_sell)#现金收入
        equity = (cumulative_sell - cumulative_buy) + pos * self.price#realized P&L+unrealized P&L
        self.data_out.equity_curve=equity
        
    def plot_curve(self):
        data = [go.Scatter(
                  x=self.data_out.equity_curve.index,
                  y=self.data_out.equity_curve.values)]
        return py.iplot(data)





In [2]:
%run macd.py

In [4]:
mb=backtest(a.macd(12,26,9),a.close)

In [5]:
mb.backtest(trade_size=100,ifshort=True)
mb.plot_curve()